In [3]:
!pip install git+https://github.com/openai/swarm.git

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-rsco93a3
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-rsco93a3
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 28.2 MB/s eta 0:00:00
  Created wheel for swarm: filename=swarm-0.1.0-py3-none-any.whl size=26001 sha256=03fcafb0d7ba

In [20]:
import os
os.environ["OPENAI_API_KEY"]="your openai api key --> goes here"

## Basic Using Demo of Swarm

In [5]:
from swarm import Swarm, Agent
client=Swarm()

agent=Agent(
    name="Basic Agent",
    instructions="You are a helpful  agent"
)

messages=[{"role":"user", "content":"Hi how are you?"}]
response=client.run(agent=agent, messages=messages)
print(response.messages[-1]["content"])

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


# Transfer Work between Agents

In [6]:
from swarm import Swarm, Agent
client=Swarm()

english_agent=Agent(
    name="English Agent",
    instructions="You only speak English"
)

french_agent=Agent(
    name="French Agent",
    instructions="You only speak French"
)


def transfer_to_french_agent():
  return french_agent


english_agent.functions.append(transfer_to_french_agent)

messages=[{"role":"user", "content":"Hi how are you?"}]
response=client.run(agent=english_agent, messages=messages)
print(response.messages[-1]["content"])

I'm doing well, thank you! How can I assist you today?


In [7]:
messages=[{"role":"user", "content":"comment vas-tu?"}]
response=client.run(agent=english_agent, messages=messages)
print(response.messages[-1]["content"])

Je vais bien, merci ! Et toi, comment vas-tu ?


## Context Variable within an Agent

In [17]:
from swarm import Swarm, Agent

client= Swarm()

def instructions(context_variable):
  name=context_variable.get("name", "User")
  return f"You are a helpful agent. Greet the user by name ({name})."


def print_school_details(context_variables:dict):
  user_id = context_variables.get("user_id", None)
  name = context_variables.get("name", None)
  school = context_variables.get("school", None)

  print(f"Account Details: {name} {user_id} {school}")
  return "Success"


agent = Agent(
    name="Agent",
    instructions=instructions,
    functions=[print_school_details]

    )


context_variable = {"name": "Starship", "user_id":123,"school":  "AdAstra"}



In [9]:
response=client.run(
    agent=agent,
    messages=[{"role":"user", "content":"Hi how are you?"}],
    context_variables=context_variable
)

print(response.messages[-1]["content"])

Hello Starship! I'm here to help you. How can I assist you today?


In [18]:
response=client.run(
    agent=agent,
    messages=[{"role":"user", "content":"Print my school name"}],
    context_variables=context_variable
)

print(response.messages[-1]["content"])

Account Details: Starship 123 AdAstra
Hello, Starship! Your school name has been printed successfully.


## Function Caling

In [19]:
from swarm import Swarm, Agent

client = Swarm()

def get_weather(location) -> str:
  return "{'temp':76 , 'unit':'F'}"


agent = Agent(
    name="Agent",
    instructions="You are a helpful agent.",
    functions=[get_weather]
)

messages = [{"role":"user", "content":"What is the weather like in San Francisco?"}]

response = client.run(agent=agent, messages=messages)
print(response.messages[-1]["content"])

The current temperature in San Francisco is 76°F.
